**В этом блокноте кратко описано использование Google BigQuery для выполнения некоторых задач по данной теме**

# План действий



## Загружаем в BigQuery исходные CSV-файлы.

- сначала загружаем CSV в google storage;
- затем создаем датасет olist и вручную через консоль создаем в нем таблицы на основе этих CSV-файлов.
- (примечание: делаем это вручную, так как это разовая задача и так быстрее. Если будет задача обрабатывать регулярно поступающие порции данных, то это можно сделать например через Pub/Sub или другими способами).

## Проверка дубликатов строк и уникальности ключевых полей

In [ ]:
# полные дубликаты строк - выдает строки, которые встречаются более 1 раза
"""
SELECT 
  COUNT(*) as num_duplicate_rows, * 
  FROM <PROJECT_ID>.olist.closed_deals
GROUP BY
  mql_id, seller_id, sdr_id, sr_id, won_date, business_segment, lead_type, lead_behaviour_profile, 
  has_company, has_gtin, average_stock, business_type, declared_product_catalog_size, declared_monthly_revenue 
HAVING num_duplicate_rows > 1;
"""

In [ ]:
# дубликаты строк по указанным столбцам - выдает строки, которые встречаются более 1 раза. Для проверки уникальности ключей
"""
SELECT 
  COUNT(*) as num_duplicate_rows,
  mql_id
  FROM <PROJECT_ID>.olist.closed_deals
GROUP BY
  mql_id
HAVING num_duplicate_rows > 1;
"""

**Уникальные ключи в таблицах:**

- closed_deals: mql_id
- customers: customer_id
- geolocation: нет уникального ключа
- marketing_qualified_leads: mql_id
- order_items: (order_id, order_item_id)
- order_payments: (order_id, payment_sequential)
- order_reviews: (review_id, order_id)
- orders: order_id или customer_id
- product_category_name_translation: product_category_name
- products: product_id
- sellers: seller_id

In [ ]:
# количество уникальных значений по указанному полю
"""
#standardSQL
SELECT
  COUNT(*) AS total_rows,
  COUNT(DISTINCT sdr_id ) AS unique_sdr,
FROM <PROJECT_ID>.olist.closed_deals ;
"""

In [ ]:
# перечень уникальных значений
"""
#standardSQL
SELECT
  DISTINCT sdr_id AS unique_sdr
FROM <PROJECT_ID>.olist.closed_deals ;
"""

## SQL-запросами объединяем таблицы и сохраняем их в качестве стационарных

**Для отладки и контроля появления ошибок при соединениях таблиц сначала делаю простые запросы, проверяя из правильность. Затем свожу их в единый запрос**

### ORDER-ITEMS

In [ ]:
# к таблице sellers добавляем средние долготу и широту по ZIP-коду
"""
#standardSQL
WITH geoavg AS (SELECT 
  geolocation_zip_code_prefix,
  AVG(geolocation_lat) AS lat,
  AVG(geolocation_lng) AS lng
FROM celestial-digit-321306.olist.geolocation
GROUP BY
    geolocation_zip_code_prefix)  


SELECT 
    s.*, 
    geoavg.lat AS sellers_geo_lat,
    geoavg.lng AS sellers_geo_lng
FROM <PROJECT_ID>.olist.sellers s LEFT JOIN geoavg ON s.seller_zip_code_prefix=geoavg.geolocation_zip_code_prefix;
"""

In [ ]:
# к таблице sellers_geo добавляем инфу из closed_deals
"""
#standardSQL
SELECT 
    s.*, 
    c.* EXCEPT(seller_id)
FROM <PROJECT_ID>.olist.sellers_geo s LEFT JOIN <PROJECT_ID>.olist.closed_deals c ON s.seller_id=c.seller_id;
"""

In [ ]:
# к order_items добавляем инфу по поставщикам (seller) и продукту (products)
"""
#standardSQL
WITH 
oit AS (SELECT 
    ord.*, 
    s.* EXCEPT(seller_id)
FROM <PROJECT_ID>.olist.order_items ord LEFT JOIN <PROJECT_ID>.olist.sellers_full s ON ord.seller_id=s.seller_id),

prod AS (SELECT 
    p.*, 
    pc.product_category_name_english AS prod_cat_en
FROM <PROJECT_ID>.olist.products p LEFT JOIN <PROJECT_ID>.olist.product_category_name_translation pc ON p.product_category_name=pc.product_category_name)

SELECT
    oit.*, 
    prod.* EXCEPT(product_id)
FROM oit LEFT JOIN prod ON oit.product_id=prod.product_id
"""

#### Итоговый запрос для создания расширенной таблицы order_items1

In [ ]:
# к таблице sellers добавляем средние долготу и широту по ZIP-коду
"""
#standardSQL
WITH geoavg AS (SELECT 
  geolocation_zip_code_prefix,
  AVG(geolocation_lat) AS lat,
  AVG(geolocation_lng) AS lng
FROM celestial-digit-321306.olist.geolocation
GROUP BY
    geolocation_zip_code_prefix),  

selzip AS (
SELECT 
    s.*, 
    geoavg.lat AS sellers_geo_lat,
    geoavg.lng AS sellers_geo_lng
FROM <PROJECT_ID>.olist.sellers s LEFT JOIN geoavg ON s.seller_zip_code_prefix=geoavg.geolocation_zip_code_prefix),

selzipclose AS (
SELECT 
    s.*, 
    c.* EXCEPT(seller_id)
FROM selzip s LEFT JOIN <PROJECT_ID>.olist.closed_deals c ON s.seller_id=c.seller_id),


oit AS (SELECT 
    ord.*, 
    s.* EXCEPT(seller_id)
FROM <PROJECT_ID>.olist.order_items ord LEFT JOIN selzipclose s ON ord.seller_id=s.seller_id),

prod AS (SELECT 
    p.*, 
    pc.product_category_name_english AS prod_cat_en
FROM <PROJECT_ID>.olist.products p LEFT JOIN <PROJECT_ID>.olist.product_category_name_translation pc ON p.product_category_name=pc.product_category_name)

SELECT
    oit.*, 
    prod.* EXCEPT(product_id)
FROM oit LEFT JOIN prod ON oit.product_id=prod.product_id
"""


### ORDER-REVIEWS

In [ ]:
# добавляем результаты обработки в Google Natural Language API и сохраняем в стационарную таблицу order_reviews2, чтобы не запускать постоянно большой запрос
"""
SELECT 
    s.*, 
    g.sent_score,
    g.sent_magnitude,
    g.entities_list,
    g.sentences_count,
    g.token_count,
    g.sentlist,
    g.tokenlist
FROM <PROJECT_ID>.olist.order_reviews s LEFT JOIN <PROJECT_ID>.olist.order_reviews_googleapi g 
ON s.review_id=g.review_id AND s.order_id=g.order_id;

"""

In [ ]:
# в order-reviews есть часть заказов, по которым оставлено 2-3 отзыва. (547 заказов), из которых с текстовыми описаниями только 244. 
# и в основном это просто история переписки по заказу, в результате которой оценка может улучшиться, ухудшиться или остаться прежней
"""
SELECT 
  COUNT(*) as num_duplicate_rows,
  SUM(LENGTH(message1)) as len_mes,
  order_id
  FROM <PROJECT_ID>.olist.order_reviews

GROUP BY
  order_id

HAVING num_duplicate_rows > 1
ORDER BY len_mes
"""
# поэтому для обработки отзывов используем следующий подход: 
# агрегируем по order-id. При этом из дат делаем 4 столбца (дата первого запроса и ответа на него, дата последнего запроса и ответа на него)
# оценку берем по последней дате. Соответственно и результаты из API google берем из последней даты: здесь логика следующая - именно последний отзыв
# можно считать как финальное состояние удовлетворенности клиента. 
# текстовые значения в title и message объединяем в строки по ордеру
# добавляем поле "количество рецензий на ордер"
# и затем полученную агрегированную таблицу добавляем к orders
# этот запрос реализует данную логику
"""
WITH tmp AS (
SELECT 
  MAX(review_answer_timestamp) as max_review_answer_date,
  order_id
  FROM `<PROJECT_ID>.olist.order_reviews2`
GROUP BY
  order_id),

score AS(
SELECT 
  s.review_score AS review_score,
  s.sent_score AS sent_score, 
  s.sent_magnitude AS sent_magnitude,
  s.entities_list AS entities_list,
  s.sentences_count AS sentences_count,
  s.token_count AS token_count,
  s.sentlist AS sentlist,
  s.tokenlist AS tokenlist,
  s.review_answer_timestamp AS review_answer_timestamp,
  s.order_id AS order_id
FROM `<PROJECT_ID>.olist.order_reviews2` s RIGHT JOIN tmp ON s.review_answer_timestamp=tmp.max_review_answer_date AND s.order_id=tmp.order_id
),

avgr AS(
SELECT 
  order_id,
  COUNT(*) as num_duplicate_rows,
  STRING_AGG(message1, " ") as message,
  STRING_AGG(title11, " ") as title,
  MIN(review_creation_date) as min_review_create_date,
  MAX(review_creation_date) as max_review_create_date,
  MIN(review_answer_timestamp) as min_review_answer_date,
  MAX(review_answer_timestamp) as max_review_answer_date
  
FROM `<PROJECT_ID>.olist.order_reviews2` 

GROUP BY
  order_id)

#HAVING num_duplicate_rows > 1)

SELECT 
avgr.*,
score.review_score,
score.sent_score AS sent_score, 
score.sent_magnitude AS sent_magnitude,
score.entities_list AS entities_list,
score.sentences_count AS sentences_count,
score.token_count AS token_count,
score.sentlist AS sentlist,
score.tokenlist AS tokenlist,
FROM avgr LEFT JOIN score ON avgr.order_id=score.order_id
"""

# и затем его присоединяем к orders:
"""
WITH tmp AS (
SELECT 
  MAX(review_answer_timestamp) as max_review_answer_date,
  order_id
  FROM `<PROJECT_ID>.olist.order_reviews2`
GROUP BY
  order_id),

score AS(
SELECT 
  s.review_score AS review_score,
  s.sent_score AS sent_score, 
  s.sent_magnitude AS sent_magnitude,
  s.entities_list AS entities_list,
  s.sentences_count AS sentences_count,
  s.token_count AS token_count,
  s.sentlist AS sentlist,
  s.tokenlist AS tokenlist,
  s.review_answer_timestamp AS review_answer_timestamp,
  s.order_id AS order_id
FROM `<PROJECT_ID>.olist.order_reviews2` s RIGHT JOIN tmp ON s.review_answer_timestamp=tmp.max_review_answer_date AND s.order_id=tmp.order_id
),

avgr AS(
SELECT 
  order_id,
  COUNT(*) as num_duplicate_rows,
  STRING_AGG(message1, " ") as message,
  STRING_AGG(title11, " ") as title,
  MIN(review_creation_date) as min_review_create_date,
  MAX(review_creation_date) as max_review_create_date,
  MIN(review_answer_timestamp) as min_review_answer_date,
  MAX(review_answer_timestamp) as max_review_answer_date
  
FROM `<PROJECT_ID>.olist.order_reviews2` 

GROUP BY
  order_id),

rev AS (
SELECT 
avgr.*,
score.review_score,
score.sent_score AS sent_score, 
score.sent_magnitude AS sent_magnitude,
score.entities_list AS entities_list,
score.sentences_count AS sentences_count,
score.token_count AS token_count,
score.sentlist AS sentlist,
score.tokenlist AS tokenlist,
FROM avgr LEFT JOIN score ON avgr.order_id=score.order_id)

SELECT 
    s.*,
    rev.* EXCEPT (order_id)
FROM `<PROJECT_ID>.olist.orders_full` s LEFT JOIN rev ON s.order_id=rev.order_id
"""

### ORDERS

In [ ]:
# к таблице customers добавляем средние долготу и широту по ZIP-коду
"""
#standardSQL
WITH geoavg AS (SELECT 
  geolocation_zip_code_prefix,
  AVG(geolocation_lat) AS lat,
  AVG(geolocation_lng) AS lng
FROM <PROJECT_ID>.olist.geolocation
GROUP BY
    geolocation_zip_code_prefix)  


SELECT 
    c.*, 
    geoavg.lat AS customers_geo_lat,
    geoavg.lng AS customers_geo_lng
FROM <PROJECT_ID>.olist.customers c LEFT JOIN geoavg ON c.customer_zip_code_prefix=geoavg.geolocation_zip_code_prefix;
"""

In [ ]:
# к orders добавляем customers_geo 
"""
#standardSQL
SELECT
    ord.*, 
    c.* EXCEPT(customer_id)
FROM <PROJECT_ID>.olist.orders ord LEFT JOIN <PROJECT_ID>.olist.customers_geo c ON ord.customer_id=c.customer_id
"""

In [ ]:
# к orders добавляем некоторые агрегированные значения из order-items
"""
#standardSQL
WITH itemavg AS (SELECT 
  order_id,
  MAX(order_item_id) AS order_item_count,
  SUM(price) AS sum_price,
  SUM(freight_value) AS sum_freight_value,
  SUM(price)+SUM(freight_value) AS sum_price_freight

FROM `<PROJECT_ID>.olist.order_items_full`
GROUP BY
    order_id)  


SELECT 
    o.*, 
    itemavg.* EXCEPT (order_id)
FROM `<PROJECT_ID>.olist.orders_customers` o LEFT JOIN itemavg ON o.order_id=itemavg.order_id;
"""

In [ ]:
# сколько заказов, в которых более одного уникального продукта
"""
WITH ordprod AS (
SELECT 
  COUNT(*) as num_duplicate_rows,
  order_id,
  product_id
  FROM <PROJECT_ID>.olist.order_items_full

GROUP BY
  order_id, 
  product_id
)

SELECT 
  COUNT(*) as num_rows,
  order_id,
  FROM ordprod

GROUP BY
  order_id

HAVING num_rows > 1
ORDER BY num_rows DESC
"""





In [ ]:
# добавляем этот показатель к orders - это заказы, для которых нет записей в order-items, т.е. нет данных о продукте и поставщике. 775 заказов
"""
WITH ordprod AS (
SELECT 
  COUNT(*) as num_duplicate_rows,
  order_id,
  product_id
  FROM <PROJECT_ID>.olist.order_items_full

GROUP BY
  order_id, 
  product_id
),

unprod AS(
SELECT 
  COUNT(*) as num_uniq_prod,
  order_id,
  FROM ordprod

GROUP BY
  order_id)

SELECT 
    o.*, 
    unprod.* EXCEPT (order_id)
FROM `<PROJECT_ID>.olist.orders_full_reviews` o LEFT JOIN unprod ON o.order_id=unprod.order_id

"""

#### Итоговый запрос для создания расширенной таблицы orders1

In [ ]:

"""
#standardSQL
WITH geoavg AS (SELECT 
  geolocation_zip_code_prefix,
  AVG(geolocation_lat) AS lat,
  AVG(geolocation_lng) AS lng
FROM `celestial-digit-321306.olist.geolocation`
GROUP BY
    geolocation_zip_code_prefix),  

custgeo AS (
SELECT 
    c.*, 
    geoavg.lat AS customers_geo_lat,
    geoavg.lng AS customers_geo_lng
FROM `<PROJECT_ID>.olist.customers` c LEFT JOIN geoavg ON c.customer_zip_code_prefix=geoavg.geolocation_zip_code_prefix),

ordcustgeo AS (
SELECT
    ord.*, 
    c.* EXCEPT(customer_id)
FROM `<PROJECT_ID>.olist.orders` ord LEFT JOIN custgeo c ON ord.customer_id=c.customer_id),

itemavg AS (SELECT 
  order_id,
  MAX(order_item_id) AS order_item_count,
  SUM(price) AS sum_price,
  SUM(freight_value) AS sum_freight_value,
  SUM(price)+SUM(freight_value) AS sum_price_freight

FROM `<PROJECT_ID>.olist.order_items1`
GROUP BY
    order_id),  

orditem1 AS (
SELECT 
    o.*, 
    itemavg.* EXCEPT (order_id)
FROM ordcustgeo o LEFT JOIN itemavg ON o.order_id=itemavg.order_id),

ordprod AS (
SELECT 
  COUNT(*) as num_prod,
  order_id,
  product_id
  FROM `<PROJECT_ID>.olist.order_items1`

GROUP BY
  order_id, 
  product_id
),

unprod AS(
SELECT 
  COUNT(*) as num_uniq_prod,
  order_id,
  FROM ordprod

GROUP BY
  order_id),

orditem2 AS (
SELECT 
    o.*, 
    unprod.* EXCEPT (order_id)
FROM orditem1 o LEFT JOIN unprod ON o.order_id=unprod.order_id),

tmp AS (
SELECT 
  MAX(review_answer_timestamp) as max_review_answer_date,
  order_id
  FROM `<PROJECT_ID>.olist.order_reviews2`
GROUP BY
  order_id),

score AS(
SELECT 
  s.review_score AS review_score,
  s.sent_score AS sent_score, 
  s.sent_magnitude AS sent_magnitude,
  s.entities_list AS entities_list,
  s.sentences_count AS sentences_count,
  s.token_count AS token_count,
  s.sentlist AS sentlist,
  s.tokenlist AS tokenlist,
  s.review_id AS review_id,
  s.review_answer_timestamp AS review_answer_timestamp,
  s.order_id AS order_id
FROM `<PROJECT_ID>.olist.order_reviews2` s RIGHT JOIN tmp ON s.review_answer_timestamp=tmp.max_review_answer_date AND s.order_id=tmp.order_id
),

avgr AS(
SELECT 
  order_id,
  COUNT(*) as num_reviews_per_order,
  STRING_AGG(message1, " ") as message,
  STRING_AGG(title11, " ") as title,
  MIN(review_creation_date) as min_review_create_date,
  MAX(review_creation_date) as max_review_create_date,
  MIN(review_answer_timestamp) as min_review_answer_date,
  MAX(review_answer_timestamp) as max_review_answer_date
  
FROM `<PROJECT_ID>.olist.order_reviews2` 

GROUP BY
  order_id),

rev AS (
SELECT 
avgr.*,
score.review_id AS review_id,
score.review_score,
score.sent_score AS sent_score, 
score.sent_magnitude AS sent_magnitude,
score.entities_list AS entities_list,
score.sentences_count AS sentences_count,
score.token_count AS token_count,
score.sentlist AS sentlist,
score.tokenlist AS tokenlist,
FROM avgr LEFT JOIN score ON avgr.order_id=score.order_id)

SELECT 
    s.*,
    rev.* EXCEPT (order_id)
FROM orditem2 s LEFT JOIN rev ON s.order_id=rev.order_id
"""

# Анализ

In [ ]:
# анализируем статус заказов
"""
WITH ordprod AS (
SELECT 
  COUNT(*) as num_duplicate_rows,
  order_id,
  product_id
  FROM <PROJECT_ID>.olist.order_items_full

GROUP BY
  order_id, 
  product_id
),

unprod AS(
SELECT 
  COUNT(*) as num_uniq_prod,
  order_id,
  FROM ordprod

GROUP BY
  order_id),

notitems AS(
SELECT 
    o.*, 
    unprod.* EXCEPT (order_id)
FROM `<PROJECT_ID>.olist.orders_full_reviews` o LEFT JOIN unprod ON o.order_id=unprod.order_id
WHERE num_uniq_prod is NULL)

SELECT 
COUNT(*) as num_po_cat,
order_status
FROM notitems 
GROUP BY order_status
"""
# основная масса заказов без указания продукта и поставщика - Недоступно и Отменено. И оценки соответственно в основном плохие
"""
Row	num_po_cat	order_status	
1	603 unavailable
2	164 canceled
3	5 created
4	1 shipped
5	2 invoiced
"""
# заказы же с указанными продуктами в основном либо доставлены, либо на разных стадиях обработки. А также есть часть отмененных
"""
Row	num_po_cat	order_status	
1	96478 delivered
2	461 canceled
3	1106 shipped
4	301 processing
5	312 invoiced
6	6 unavailable
7	2 approved
"""
# т.е. статус заказа сильно влияет на вероятность получения плохой оценки